In [1]:
data = spark.read \
            .option("multiLine", True) \
            .csv('/home/ilyes/M2/algortihmique_BD/archive(2)/mental_health.csv',inferSchema=True, header=True)

In [2]:
data.printSchema()

root
 |-- text: string (nullable = true)
 |-- label: integer (nullable = true)



In [3]:
data.head()

Row(text='dear american teens question dutch person heard guys get way easier things learn age us sooooo thth graders like  right guys learn math', label=0)

In [4]:
from pyspark.sql.functions import regexp_replace, trim

data = data.withColumn('text', regexp_replace('text', '[\s]{2,}', ''))
data = data.withColumn('text', trim(data.text))

In [5]:
from pyspark.sql.functions import col, lower
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover

In [6]:
data = data.withColumn("text", lower(col('text')))
data = data.filter(data.text != '')

In [7]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(data)

In [8]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
Nstopwords = remover.transform(wordsData)

In [9]:
from pyspark.ml.feature import Word2Vec

word2Vec = Word2Vec(vectorSize=150, inputCol="filtered", outputCol="features")
word2vec_data = word2Vec.fit(Nstopwords)

In [34]:
from pyspark.ml.feature import Word2VecModel

In [33]:
word2vec_data.save('/home/hdoop/w2v')

In [10]:
w2v_data = word2vec_data.transform(Nstopwords)

In [11]:
# split dataframes between 0s and 1s
zeros = w2v_data.filter(w2v_data["label"]==0)
ones = w2v_data.filter(w2v_data["label"]==1)
# split datasets into training and testing
train0, test0 = zeros.randomSplit([0.8,0.2])
train1, test1 = ones.randomSplit([0.8,0.2])
# merge datasets back together
train = train0.union(train1)
test = test0.union(test1)

In [12]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [22]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier()

dtparamGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [2, 5, 10, 20])
             .addGrid(dt.maxBins, [10, 20, 40])
             .build())

In [23]:
crossvaldt = CrossValidator(estimator=dt,
                          estimatorParamMaps=dtparamGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=10)
cvModeldt = crossvaldt.fit(train)

In [24]:
tested = cvModeldt.transform(test)
evaluation = BinaryClassificationEvaluator() #AUC
evaluation.evaluate(tested)

0.8903006838647619

In [25]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()

paramGridlr = ParamGridBuilder() \
    .addGrid(lr.elasticNetParam, [0, 1]) \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()
    
crossvallr = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGridlr,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=10)
cvModellr = crossvallr.fit(train)

In [26]:
tested = cvModellr.transform(test)
evaluation = BinaryClassificationEvaluator() #AUC
evaluation.evaluate(tested)

0.9536660891085791

In [20]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier()

rfparamGrid = (ParamGridBuilder()
               .addGrid(rf.maxDepth, [2, 5, 10])
               .addGrid(rf.maxBins, [5, 10, 20])
               .addGrid(rf.numTrees, [5, 20, 50])
             .build())

rfcv = CrossValidator(estimator = rf,
                      estimatorParamMaps = rfparamGrid,
                      evaluator = BinaryClassificationEvaluator(),
                      numFolds = 10)
cvModelrf = rfcv.fit(train)

In [22]:
best = cvModelrf.bestModel

In [29]:
best.save('/home/hdoop/model/')

In [30]:
best._java_obj.getMaxDepth()

10

In [31]:
best._java_obj.getMaxBins()

10

In [32]:
best._java_obj.getNumTrees()

50

In [21]:
tested = cvModelrf.transform(test)
evaluation = BinaryClassificationEvaluator() #AUC
evaluation.evaluate(tested)

0.9636242814198899

In [18]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier()

gbtparamGrid = ParamGridBuilder().build()

gbtcv = CrossValidator(estimator = gbt,
                      estimatorParamMaps = gbtparamGrid,
                      evaluator = BinaryClassificationEvaluator(),
                      numFolds = 10)
cvModelgbt = gbtcv.fit(train)

In [19]:
tested = cvModelgbt.transform(test)
evaluation = BinaryClassificationEvaluator() #AUC
evaluation.evaluate(tested)

0.9588292238991081

In [14]:
from pyspark.ml.classification import LinearSVC

svc = LinearSVC()

svcparamGrid = (ParamGridBuilder()
                .addGrid(svc.maxIter, [10, 100])
                .addGrid(svc.regParam, [0.001, 0.01, 1.0,10.0])
                .build())

svccv = CrossValidator(estimator = svc,
                      estimatorParamMaps = svcparamGrid,
                      evaluator = BinaryClassificationEvaluator(),
                      numFolds = 10)
cvModelsvc = svccv.fit(train)

In [19]:
bestmodel = cvModelsvc.bestModel
bestmodel._java_obj.getRegParam()

0.001

In [15]:
tested = cvModelsvc.transform(test)
evaluation = BinaryClassificationEvaluator() #AUC
evaluation.evaluate(tested)

0.9600915541100232

In [28]:
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=92769)
featurizedData = hashingTF.transform(Nstopwords)

In [29]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
tf_idf_data = idfModel.transform(featurizedData)

In [70]:
data.filter(data.label==0).count()

14138

In [71]:
data.filter(data.label==1).count()

13837

In [30]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()

In [31]:
paramGrid = ParamGridBuilder() \
    .addGrid(lr.elasticNetParam, [0, 1]) \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

In [32]:
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=10)
cvModel = crossval.fit(train)

In [33]:
tested = cvModel.transform(test)

In [34]:
evaluation = BinaryClassificationEvaluator() #AUC
evaluation.evaluate(tested)

0.9551031317760491

In [35]:
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes(modelType="multinomial")

nbparamGrid = (ParamGridBuilder()
               .addGrid(nb.smoothing, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
               .build())

In [36]:
crossvalnb = CrossValidator(estimator=nb,
                          estimatorParamMaps=nbparamGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=10)
cvModelnb = crossvalnb.fit(train)

Py4JJavaError: An error occurred while calling o9024.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2537.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2537.0 (TID 4653) (172.23.0.86 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (NaiveBayes$$Lambda$4329/0x0000000841866840: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:136)
	at org.apache.spark.sql.catalyst.expressions.ScalaUDF.eval(ScalaUDF.scala:1196)
	at org.apache.spark.ml.stat.SummaryBuilderImpl$MetricsAggregate.update(Summarizer.scala:382)
	at org.apache.spark.ml.stat.SummaryBuilderImpl$MetricsAggregate.update(Summarizer.scala:345)
	at org.apache.spark.sql.catalyst.expressions.aggregate.TypedImperativeAggregate.upd

In [92]:
tested = cvModelnb.transform(test)
evaluation = BinaryClassificationEvaluator() #AUC
evaluation.evaluate(tested)

0.4761694294819699

In [13]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

layers = [262144, 174762, 1]

trainer = MultilayerPerceptronClassifier(layers=layers)
model = trainer.fit(train)

Py4JJavaError: An error occurred while calling o158.fit.
: java.lang.NegativeArraySizeException: -1431481003
	at scala.reflect.ManifestFactory$DoubleManifest.newArray(Manifest.scala:194)
	at scala.reflect.ManifestFactory$DoubleManifest.newArray(Manifest.scala:191)
	at breeze.linalg.DenseVector$.zeros$mDc$sp(DenseVector.scala:274)
	at org.apache.spark.ml.ann.FeedForwardModel$.apply(Layer.scala:601)
	at org.apache.spark.ml.ann.FeedForwardTopology.model(Layer.scala:422)
	at org.apache.spark.ml.ann.FeedForwardTrainer.train(Layer.scala:844)
	at org.apache.spark.ml.classification.MultilayerPerceptronClassifier.$anonfun$train$1(MultilayerPerceptronClassifier.scala:228)
	at org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:191)
	at scala.util.Try$.apply(Try.scala:213)
	at org.apache.spark.ml.util.Instrumentation$.instrumented(Instrumentation.scala:191)
	at org.apache.spark.ml.classification.MultilayerPerceptronClassifier.train(MultilayerPerceptronClassifie

In [ ]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier()
gbtclassifier = gbt.fit(train)